In [1]:
# --- Imports ---
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct" 
CSV_PATH = r"C:\Users\stdFurqan\Downloads\urdu_v1\test.csv" 
TEXT_COL = "Cleaned_Tweet"
LABEL_COL = "Class"
LABELS = ["P", "N"]
SEED = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Reproducibility ---
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

In [3]:
# ✅ Local path where model is stored
MODEL_PATH = r"C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b)"

print(f"🧠 Loading model from {MODEL_PATH} ...")

# ✅ Load tokenizer from local folder
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ✅ Load model from local folder
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,   # ✅ best for 40-series
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

print("✅ Model loaded successfully from local directory!")

🧠 Loading model from C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b) ...


`torch_dtype` is deprecated! Use `dtype` instead!
The module name LAMA_3_dot_2(1b) (originally LAMA_3.2(1b)) is not a valid Python identifier. Please rename the original module to avoid import issues.


✅ Model loaded successfully from local directory!


In [4]:
# --- Load dataset ---
df = pd.read_csv(CSV_PATH)
# ✅ Clean up any extra spaces or tabs in column names
df.columns = df.columns.str.strip()
print(df.columns.tolist())


['Cleaned_Tweet', 'Class']


In [5]:




texts = df[TEXT_COL].astype(str).tolist()
true_labels = df[LABEL_COL].astype(str).tolist()

PROMPT_TEMPLATE = (
    "You are an intelligent Urdu language assistant."
    "Classify the following news headline into one of these categories: Positive, or Negative based on the examples below.\n\n"
    
    "=== Positive Examples ===\n"
    "1 Text: کلاسک ویڈیو گیم کی شکل میں بنایا گیا منفرد کیک نیویارک کیک خواہ سالگرہ کا ہویا کسی اور\n   Label: Positive\n"
    "2 Text: مجاہد اعظم مولانا غلام اعظم نے اپنی جوانی سے بڑھا پے تک صبر عزمیت اور مسلسل جدوجہد کی تاریخ رقم کر دی اس\n   Label: Positive\n"
    "3 Text: میں آپکی بات سے اتفاق کرتا ہوں لیکن میں سب کی رائے لینا اور سننا پسند کرتا ہوں ایک عادی بھائی ہیں کبھی کام کی بات نئ کی پ سنتا\n   Label: Positive\n"
    "4 Text: وینا ملک کا بیٹا سوشل میڈیا پر مقبولیت میں شلپا شیٹھی کے بیٹے سے آگے کراچی وینا ملک کے بیٹے ابرام خان\n   Label: Positive\n\n"

    "=== Negative Examples ===\n"
    "5 Text: سیالکوٹ ہندوستانی فورسز کی بلا اشتعال فائرنگ پاکستان کے صوبہ پنجاب کے شہر سیالکوٹ کی ورکنگ\n   Label: Negative\n"
    "6 Text: کراچی میں تمام نیوز چینل کی بندشایم کیو ایم اور پاکستان مسلم لیگ زندہ بادپاکستان ملک ہے شہنشاہوں کا بدمعاشوں کا\n   Label: Negative\n"
    "7 Text: ﻋﺎﻟﻤﯽ ﺍﺳﺘﻌﻤﺎﺭ ﺍﻣﺮﯾﮑﺎ ﻭ ﺍﺳﺮﺍﺋﯿﻞ ﺍﻭﺭﺍ ﻧﮑﮯ ﻧﻤﮏ ﺧﻮﺍﺭ ﺳﻌﻮﺩﯼ ﻋﺮﺏ ﮐﮯ ﺳﯿﻨﮯﻣﯿﮟ ﺧﻨﺠﺮ ﮐﯽ ﻣﺎﻧﻨﺪ ﻣﺎﮦ ﻣﺤﺮﻡ ﺻﻔﺮﺍﻭﺭ ﺭﺑﯿﻊ ﺍﻻﻭﻝ\n   Label: Negative\n"
    "8 Text: جناب اسپیکر آج سے نون لیگ کو منافق لیگ کا نام دیا جائے کیونکہ نون لیگ نے سچ تو کبھی بولنا نہیں\n   Label: Negative\n\n" 

    "Reply with only one word: P, or N.\n"
    "Now classify the following text in the same way.\n\n"
    "Headline: {text}\n\nLabel:"
)


pattern = re.compile(r"\b(P|N)\b", re.IGNORECASE)

def extract_label(output):
    m = pattern.search(output)
    return m.group(1).capitalize() if m else "Unknown"

pred_labels = []
batch_size = 4  # smaller batch to fit in T4 VRAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    prompts = [PROMPT_TEMPLATE.format(text=t) for t in batch]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for prompt, full_out in zip(prompts, decoded):
        gen = full_out[len(prompt):].strip() if full_out.startswith(prompt) else full_out
        pred_labels.append(extract_label(gen))

# --- Evaluation ---
report = classification_report(true_labels, pred_labels, labels=LABELS, digits=4, zero_division=0)
acc = accuracy_score(true_labels, pred_labels)
print("\n📊 Classification Report:\n")
print(report)
print(f"Accuracy: {acc:.4f}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📊 Classification Report:

              precision    recall  f1-score   support

           P     0.7188    0.1597    0.2614       144
           N     0.3333    0.0067    0.0131       150

   micro avg     0.6857    0.0816    0.1459       294
   macro avg     0.5260    0.0832    0.1372       294
weighted avg     0.5221    0.0816    0.1347       294

Accuracy: 0.0816
